In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")

In [3]:
import os
import sys

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 8)
import mne
mne.set_log_level('ERROR')

from braindecode.datasets import MOABBDataset
from braindecode.datasets.tuh import TUHAbnormal
from braindecode.datautil.preprocess import filterbank, preprocess, Preprocessor
from braindecode.datautil.windowers import create_windows_from_events, create_fixed_length_windows

sys.path.insert(0, '/home/gemeinl/code/braindecode-features/')
from braindecode_features import extract_windows_ds_features, save_features, filter_df

In [4]:
ds_name = 'TUHAbnormal'  # 'Schirrmeister2017'
out_path = None  # './tmp/'#'/home/lukas/Code/HGD/'
n_jobs = 2
agg_func = None#'mean'
if ds_name in ['Schirrmeister2017', 'BNCI14001']:
    subject_id = 1  
else:
    assert ds_name in ['TUHAbnormal']
    recording_ids = list(range(20))

#------------------------------------------------------------------------------
# stuffs to optimize
sfreq = 100  # 250
# original trials have 4s duration
frequency_bands = [(4,8), (8, 13)]
trial_start_offset_samples = 0  # int(0.5*sfreq)
window_size_samples = 500
window_stride_samples = 500

# clf + hyperparams
#------------------------------------------------------------------------------
factor = 1e6
max_abs_val = 800
trial_stop_offset_samples = None
sensors = {
    'Schirrmeister2017': (
        'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CCP1h', 'CCP2h', 'CCP3h', 'CCP4h',
        'CCP5h', 'CCP6h', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'CPP1h',
        'CPP2h', 'CPP3h', 'CPP4h', 'CPP5h', 'CPP6h', 'CPz', 'Cz', 'FC1', 'FC2',
        'FC3', 'FC4', 'FC5', 'FC6', 'FCC1h', 'FCC2h', 'FCC3h', 'FCC4h',
        'FCC5h', 'FCC6h', 'FCz', 'FFC1h', 'FFC2h', 'FFC3h', 'FFC4h', 'FFC5h',
        'FFC6h'
    ),
    'BNCI2014001': (
        'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'CP1', 'CP2', 'CP3', 'CP4', 'CPz',
        'Cz', 'FC1', 'FC2', 'FC3', 'FC4', 'FCz', 'Fz', 'P1', 'P2', 'POz', 'Pz'
    ),
    'TUHAbnormal': (
        'EEG A1-REF', 'EEG A2-REF', 'EEG C3-REF', 'EEG C4-REF',
        'EEG CZ-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG F7-REF',
        'EEG F8-REF', 'EEG FP1-REF', 'EEG FP2-REF', 'EEG FZ-REF',
        'EEG O1-REF', 'EEG O2-REF', 'EEG P3-REF', 'EEG P4-REF',
        'EEG PZ-REF', 'EEG T1-REF', 'EEG T2-REF', 'EEG T3-REF',
        'EEG T4-REF'
    ),
}
sensors = sensors[ds_name]
train_eval_split = {
    'Schirrmeister2017': 'run', 
    'BNCI2014001': 'session',
    'TUHAbnormal': 'train',
}
eval_name = {
    'Schirrmeister2017': 'test',
    'BNCI2014001': 'session_E',
    'TUHAbnormal': 'False',
}

In [5]:
if ds_name in ['Schirrmeister2017', 'BNCI14001']:
    ds = MOABBDataset(
        dataset_name=ds_name,
        subject_ids=[subject_id],
    )
else:
    assert ds_name in ['TUHAbnormal']
    ds = TUHAbnormal(
        path='/data/schirrmr/gemeinl/tuh-abnormal-eeg/raw/v2.0.0/edf/',
        recording_ids=recording_ids,
        target_name='pathological',
        preload=False,
        add_physician_reports=True,
    )

In [6]:
split_indicator = train_eval_split[ds_name]
eval_ds_name = eval_name[ds_name]

In [7]:
# split into train and test session
splits = ds.split(split_indicator)
for split_name, split_ds in splits.items():
    if split_name == eval_ds_name:
        continue
    break
print(split_name)

True


In [8]:
if ds_name in ['TUHAbnormal']:
    split_name = 'eval' if split_name == 'False' else 'train'

In [9]:
# some preprocessing steps
preprocessors = [
    Preprocessor(
        apply_on_array=False,
        fn='pick_channels', 
        ch_names=sensors, 
        ordered=True,
    ),
    Preprocessor(
        apply_on_array=True,
        fn=lambda x: x * factor,
    ),
    Preprocessor(
        apply_on_array=True,
        fn=lambda x: np.clip(x, -max_abs_val, max_abs_val),
    ),
    Preprocessor(
        apply_on_array=False,
        fn='resample', 
        sfreq=sfreq,
    ),
]
# Additional preprocessing for TUHAbnormal
if ds_name in ['TUHAbnormal']:
    preprocessors.extend([
        # EEG O1-REF -> O1
        Preprocessor(
            apply_on_array=False,
            fn='rename_channels',
            mapping=lambda ch: ch[ch.find(' ')+1:ch.find('-')],
        ),
        # discard first 60s
        Preprocessor(
            apply_on_array=False,
            fn='crop',
            tmin=60,
        ),
    ])
preprocessors.append(
    # used by connectivity, time, and cross-frequency domain features
    # not used by dft, cwt domain features
    Preprocessor(
        apply_on_array=False,
        fn=filterbank, 
        frequency_bands=sorted(frequency_bands, key=lambda b: b[0]), 
        drop_original_signals=False, 
    )
)

In [10]:
# apply some preprocessing
preprocess(
    concat_ds=split_ds,
    preprocessors=preprocessors,
)
# extract compute windows 
if ds_name in ['Schirrmeister2017', 'BNCI2014001']:
    create_windows = create_windows_from_events
    kwargs = {}
else:
    assert ds_name in ['TUHAbnormal']
    create_windows = create_fixed_length_windows
    # map boolean pathological targets to integer 
    kwargs = {'mapping': {False: 0, True: 1}}
windows_ds = create_windows(
    concat_ds=split_ds,
    start_offset_samples=trial_start_offset_samples,
    stop_offset_samples=trial_stop_offset_samples,
    drop_last_window=False,
    window_size_samples=window_size_samples,
    window_stride_samples=window_stride_samples,
    **kwargs,
)

In [11]:
for x, y, ind in windows_ds:
    break
x.shape, y, ind

((63, 500), 0, [0, 6000, 6500])

In [12]:
df = extract_windows_ds_features(
    windows_ds=windows_ds,
    frequency_bands=frequency_bands,
    n_jobs=n_jobs,
    params={'Time__higuchi_fractal_dimension__kmax': 3},
    domains=None,
)

INFO:braindecode_features.feature_extraction:Computing features of domain: Hilbert.
INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:braindecode_features.feature_extraction:Computing features of domain: Cross-frequency.
INFO:braindecode_features.feature_extraction:Computing features of domain: Wavelet.
INFO:braindecode_features.feature_extraction:Computing features of domain: Fourier.
INFO:braindecode_features.feature_extraction:Computing features of domain: Time.


In [13]:
df

Domain    Description                     Time  ...             Hilbert  \
Feature         Trial Window Target covariance  ... phase_locking_value   
Channel                                  A1-A2  ...               T1-T4   
Frequency                                  4-8  ...                8-13   
0                   0      0      0  35.343983  ...            0.592420   
1                   0      1      0  26.628353  ...            0.656590   
2                   0      2      0  29.428747  ...            0.725725   
3                   0      3      0  96.053429  ...            0.857858   
4                   0      4      0  72.775475  ...            0.744106   
...               ...    ...    ...        ...  ...                 ...   
1550                4    267      1   4.301175  ...            0.496300   
1551                4    268      1   5.686838  ...            0.608351   
1552                4    269      1   3.335802  ...            0.218010   
1553                4    270      1   4.507133  ...            0.575744   
1554                4    271      1   6.496952  ...            0.485992   

Domain                                   
Feature                                  
Channel       T2-T3     T2-T4     T3-T4  
Frequency      8-13      8-13      8-13  
0          0.532948  0.846600  0.441852  
1          0.465723  0.865962  0.508563  
2          0.666351  0.887772  0.593228  
3          0.756994  0.921302  0.805921  
4          0.740304  0.888831  0.726527  
...             ...       ...       ...  
1550       0.361156  0.881787  0.326279  
1551       0.448643  0.841559  0.514064  
1552       0.212520  0.716193  0.182217  
1553       0.443076  0.823722  0.451964  
1554       0.334793  0.808300  0.350087  

[1555 rows x 2292 columns]

In [14]:
df.dtypes

Domain       Feature              Channel  Frequency
Description  Trial                                        int64
             Window                                       int64
             Target                                       int64
Time         covariance           A1-A2    4-8          float32
                                  A1-C3    4-8          float32
                                                         ...   
Hilbert      phase_locking_value  T1-T3    8-13         float32
                                  T1-T4    8-13         float32
                                  T2-T3    8-13         float32
                                  T2-T4    8-13         float32
                                  T3-T4    8-13         float32
Length: 2292, dtype: object

In [19]:
out_path = None
# save each trial feature matrix (windows x features) to an individual file
if out_path is not None:
    if ds_name in ['Schirrmeister2017', 'BNCI2014001']:
        out_path = os.path.join(out_path, str(subject_id), split_name)
    else:
        assert ds_name in ['TUHAbnormal']
        out_path = os.path.join(out_path, split_name)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    save_features(
        df=df, 
        out_path=out_path, 
    )

In [21]:
# inspect subsets of the feature data frame
filter_df(
    df=df, 
    query='Description', 
    exact_match=False, 
    level_to_consider=None,
)

Domain    Description              
Feature         Trial Window Target
Channel                            
Frequency                          
0                   0      0      0
1                   0      1      0
2                   0      2      0
3                   0      3      0
4                   0      4      0
...               ...    ...    ...
1550                4    267      1
1551                4    268      1
1552                4    269      1
1553                4    270      1
1554                4    271      1

[1555 rows x 3 columns]